# Object Detection

Note: some code in this notebook is taken or derived from the manual provided for the university course "data mining" at UU.

## Explanation of choice for features, plan for tackling the dimensionality

As I already used gender for the other question, I decided on using object detection to compare the movie trailers over the different years.
Based on the objects in the movie trailers of the years 1920-1940, 1960-1980 and 2000-2020, I want to investigate if the objects in movies change a lot over time or if they stay quite similar. Further, from looking at the most common objects in movies over the different decades, I want to explore if the there are any objects that 'stick out' in comparison with the other years. Other than that, I expect that objects like 'laptop' and 'cell phone', 'keyboard' and 'skyscraper' to be more common in recent years (2000-2020).

From exercise 5.2 I had subsets of 5 movie trailers for the years 1920-1940, 1960-1980 and 2000-2020.
I used coco classes for object detection, as this at least for some sample frames I tried was better/more accurate.
To tackle the dimensionality of the data, I used the tool 'scenedetect' with a threshold of 20 to divide each trailer in scenes. I chose this threshold because with a higher threshold a lot of trailers are made up of only one scene, and a lower threshold may be difficult to manage computationally.
Further, I only used on frame per scene (I took the frame in the middle of a scene), to keep the number of frames at a computationally manageable level. 
Each frame can contain multiple objects. I wrote a loop with for each detected coco class in an image (I am using the indexes of the classes for this), 1 is added to the count of this class. In this way, I can include all objects found in the frames in my analysis. I increased the threshold of the object detection to 0.7, as I want the model to be at least 70% sure that this object is actually the object from the coco class.
Further, for comparing the coco classes, I took the mean of each class for each of the years (1920-1940, 1960-1980 and 2000-2020), and normalized these means by the number of frames of each of the years. In the comparison, I compare the top 15 coco classes of each years using these normalized means. The normalization is necessary because the number of frames varies quite a lot for the different years (52 to 344), and more frames lead to a higher number of detected objects.

Suppress warnings and info loggings

In [81]:
import logging, sys
logging.disable(sys.maxsize)

In [59]:
import warnings
warnings.filterwarnings('ignore')

Importing libraries

In [1]:
import numpy as np
import pandas as pd
import cv2

from scenedetect import VideoManager
from scenedetect import SceneManager

from scenedetect.detectors import ContentDetector

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import tensorflow_hub as hub
from tensorflow.keras.preprocessing import image

import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from urllib.request import urlopen
from tqdm.notebook import tqdm

Function to detect scences in the videos

In [5]:
def find_scenes(video_path, threshold=20.0):
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(
        ContentDetector(threshold=threshold))
    base_timecode = video_manager.get_base_timecode()
    video_manager.set_downscale_factor()
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager, show_progress=False)    
    return scene_manager.get_scene_list(base_timecode)

This function takes the path and name of a video and returns a dataframe including the name of the video ('video'),its path ('path'), the scene ('scene') and the name of the image frame ('frame'). The framge is the image in the middle of a scene.

In [58]:
def process_video(video_path, filename):
    
    scene_list = find_scenes(video_path) # changed
    
    
    cap = cv2.VideoCapture(video_path) #Video capturing from video files

    frames = []
    shot_length = []
    frame_name = []
    
    i = 0

    for start_time, end_time in scene_list:
        i = i+1
        duration = end_time - start_time
        frame = (start_time.get_frames() + int(duration.get_frames() / 2))
        cap.set(cv2.CAP_PROP_POS_FRAMES,frame)
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imwrite("2000_2020/"+str(filename)+str(i)+".jpeg", frame) #change folder for each year
        frames.append(frame)
        frame_name.append("2000_2020/"+str(filename)+str(i)+".jpeg") #change folder for each year
    
    df = pd.DataFrame()
    df['video'] = [filename for i in range(len(frames))]
    df['path'] = [video_path for i in range(len(frames))]
    df['scene'] = range(len(frames))
    df['frame'] = frame_name
                  
    
    return df

## 1920-1940

Get the name of the trailers in my subset, in a list

In [76]:
from os import listdir
from os.path import isfile, join

mypath = '/Users/arleenlindenmeyer/Desktop/ADS/data_mining/exam2/1920_1940/'

filenames_20_40 = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [77]:
filenames_20_40

['ModernTimesTrailer.mp4',
 'LibeledLadyTrailer.mp4',
 'LittleCaesarTrailer.mp4',
 'NothingSacredTrailer.mp4',
 'EvelynPrenticTrailer.mp4']

Get a dataframe with the name of the trailer ('video'), its path ('path'), the scene ('scene') and the name of the image frame ('frame') for each scene.

In [82]:
list_dfs_20_40 = []

for i in filenames_20_40:
    filename = i
    videopath = '/Users/arleenlindenmeyer/Desktop/ADS/data_mining/exam2/1920_1940/'+i
    df = process_video(video_path= videopath, filename=filename)
    list_dfs_20_40.append(df)

In [113]:
df_20_40 = pd.concat(list_dfs_20_40)
df_20_40.head()

,video,path,scene,frame
0,ModernTimesTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,0,2000_2020/ModernTimesTrailer.mp41.jpeg
1,ModernTimesTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,1,2000_2020/ModernTimesTrailer.mp42.jpeg
2,ModernTimesTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,2,2000_2020/ModernTimesTrailer.mp43.jpeg
3,ModernTimesTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,3,2000_2020/ModernTimesTrailer.mp44.jpeg
4,ModernTimesTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,4,2000_2020/ModernTimesTrailer.mp45.jpeg


In total 52 frames/scenes in the years 1920-1940

In [84]:
len(df_20_40)

52

Load the object detection model

In [34]:
hub_model = hub.load("/Users/arleenlindenmeyer/Desktop/ADS/data_mining/exam2/faster_rcnn_resnet50_v1_640x640_1")


INPUT_SHAPE = (224, 224) 

Loading the coco classes

In [30]:
url_classes = 'https://raw.githubusercontent.com/nightrome/cocostuff/master/labels.txt'
coco_classes = [str(x)[str(x).find(' '):].strip().replace("'", '')
                for x in urlopen(url_classes).read().splitlines()]

In [31]:
len(coco_classes)

183

Function to load the image frame from the dataframe path

In [32]:
def load_image_from_path(image_path, target_size=None, color_mode='rgb'):
    pil_image = image.load_img(image_path, 
                               target_size=target_size,
                            color_mode=color_mode)
    return image.img_to_array(pil_image)

Function outputs for each frame, the coco classes as integers (e.g., if there are 2 dogs and dog is coco class 18, then 18. = 2) in lists. I used a threshold of 0.7 because I want the model to be at least quite sure that the object is there/this exact object.

In [35]:
all_classes = []

for file in tqdm(df_20_40.frame): # changed for each years
    
    classes = [0] * 183
    
    color_image = load_image_from_path(file, color_mode='rgb', target_size=(INPUT_SHAPE)).astype('uint8')
    color_image_batched = np.expand_dims(color_image, axis=0) 

    results = hub_model(color_image_batched) # input the image to the model
    
    detection_scores = results['detection_scores'].numpy()[0] 
    detection_classes = results['detection_classes'].numpy()[0]


    indices = np.where(detection_scores > 0.7) # only select labels with p > 0.7
    detection_classes_sel = detection_classes[indices] # get the classes where p > 0.7
    
    detection_classes_sel_list = detection_classes_sel.astype(np.int32).tolist()
    
    
    
    for i in detection_classes_sel_list:    # iterate over list, get index
        classes[i] += 1 # add 1 on this specific index   
    
    all_classes.append(classes)

  0%|          | 0/52 [00:00<?, ?it/s]

Dropping the index

In [114]:
df_20_40.reset_index(drop=True, inplace=True)

Creating the dataframe for the coco classes

In [100]:
df_classes_20_40 = pd.DataFrame(all_classes, columns=coco_classes)

Dropping the index

In [101]:
df_classes_20_40.reset_index(drop=True, inplace=True)
df_classes_20_40.head()

,unlabeled,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,...,wall-other,wall-panel,wall-stone,wall-tile,wall-wood,water-other,waterdrops,window-blind,window-other,wood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Concatenating the dataframes including the scenes/frames with the coco classes dataframe

In [103]:
df_20_40_classes = pd.concat([df_20_40, df_classes_20_40], axis=1)
df_20_40_classes.head()

,video,path,scene,frame,unlabeled,person,bicycle,car,motorcycle,airplane,...,wall-other,wall-panel,wall-stone,wall-tile,wall-wood,water-other,waterdrops,window-blind,window-other,wood
0,ModernTimesTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,0,2000_2020/ModernTimesTrailer.mp41.jpeg,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ModernTimesTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,1,2000_2020/ModernTimesTrailer.mp42.jpeg,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ModernTimesTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,2,2000_2020/ModernTimesTrailer.mp43.jpeg,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ModernTimesTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,3,2000_2020/ModernTimesTrailer.mp44.jpeg,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ModernTimesTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,4,2000_2020/ModernTimesTrailer.mp45.jpeg,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
df_20_40_classes = df_20_40_classes.iloc[:, 4:]

Getting the mean of for each coco class

In [115]:
mean_classes_20_40 = df_20_40_classes.mean(axis=0)
mean_classes_20_40 = pd.DataFrame(mean_classes_20_40, columns=['mean'])

Normalizing the means by amount of frames, and showing the top 10 coco classes in the years 1920-1940

In [111]:
normalized_mean_classes_20_40 = mean_classes_20_40['mean'] / len(df_20_40_classes)
normalized_mean_classes_20_40 = pd.DataFrame(normalized_mean_classes_20_40, columns=['mean'])
normalized_mean_classes_20_40.sort_values(ascending= False, by = 'mean').head(10)

,mean
person,0.030325
tie,0.001109
chair,0.001109
clock,0.000740
tv,0.000740
parking meter,0.000370
boat,0.000370
wine glass,0.000370
laptop,0.000370
car,0.000370


## 1960-1980

Repeat process above for the years 1960-1980

In [116]:
mypath = '/Users/arleenlindenmeyer/Desktop/ADS/data_mining/exam2/1960_1980/'

filenames_60_80 = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [117]:
filenames_60_80

['ChildrenOfTheDamnedTrailer.mp4',
 'LadySnowblood2Trailer.mp4',
 'TheScalphuntersTrailer.mp4',
 'ZForZachariahTrailer.mp4',
 'KhartoumTrailer.mp4']

In [118]:
list_dfs_60_80 = []

for i in filenames_60_80:
    filename = i
    videopath = '/Users/arleenlindenmeyer/Desktop/ADS/data_mining/exam2/1960_1980/'+i
    df = process_video(video_path= videopath, filename=filename)
    list_dfs_60_80.append(df)

In [119]:
df_60_80 = pd.concat(list_dfs_60_80)

In [120]:
df_60_80.reset_index(drop=True, inplace=True)
df_60_80.head()

,video,path,scene,frame
0,ChildrenOfTheDamnedTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,0,2000_2020/ChildrenOfTheDamnedTrailer.mp41.jpeg
1,ChildrenOfTheDamnedTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,1,2000_2020/ChildrenOfTheDamnedTrailer.mp42.jpeg
2,ChildrenOfTheDamnedTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,2,2000_2020/ChildrenOfTheDamnedTrailer.mp43.jpeg
3,ChildrenOfTheDamnedTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,3,2000_2020/ChildrenOfTheDamnedTrailer.mp44.jpeg
4,ChildrenOfTheDamnedTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,4,2000_2020/ChildrenOfTheDamnedTrailer.mp45.jpeg


In [121]:
len(df_60_80)

299

In [122]:
all_classes = []

for file in tqdm(df_60_80.frame): # change for each years
    
    classes = [0] * 183
    
    color_image = load_image_from_path(file, color_mode='rgb', target_size=(INPUT_SHAPE)).astype('uint8')
    color_image_batched = np.expand_dims(color_image, axis=0) 

    results = hub_model(color_image_batched) # input the image to the model
    
    detection_scores = results['detection_scores'].numpy()[0]
    detection_classes = results['detection_classes'].numpy()[0]


    indices = np.where(detection_scores > 0.7) # only select labels with p > 0.7
    detection_classes_sel = detection_classes[indices] # get the classes where p > 0.7
    
    detection_classes_sel_list = detection_classes_sel.astype(np.int32).tolist()
    
    
    
    for i in detection_classes_sel_list:    # iterate over list, get index
        classes[i] += 1 # add 1 on this specific index   
    
    all_classes.append(classes)

  0%|          | 0/299 [00:00<?, ?it/s]

In [123]:
df_classes_60_80 = pd.DataFrame(all_classes, columns=coco_classes)

In [124]:
df_classes_60_80.reset_index(drop=True, inplace=True)
df_classes_60_80.head()

,unlabeled,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,...,wall-other,wall-panel,wall-stone,wall-tile,wall-wood,water-other,waterdrops,window-blind,window-other,wood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
df_60_80_classes = pd.concat([df_60_80, df_classes_60_80], axis=1)

In [127]:
df_60_80_classes = df_60_80_classes.iloc[:, 4:]

In [128]:
mean_classes_60_80 = df_60_80_classes.mean(axis=0)
mean_classes_60_80 = pd.DataFrame(mean_classes_60_80, columns=['mean'])

In [162]:
normalized_mean_classes_60_80 = mean_classes_60_80['mean'] / len(df_60_80_classes)
normalized_mean_classes_60_80 = pd.DataFrame(normalized_mean_classes_60_80, columns=['mean'])
normalized_mean_classes_60_80.sort_values(ascending= False, by = 'mean').head(10)

,mean
person,0.003076
horse,0.000459
tv,0.000224
dog,0.000179
clock,0.000145
cow,0.000101
parking meter,0.000089
bird,0.000078
tie,0.000056
bottle,0.000045


## 2000-2020

Repeat process above for the years 2000-2020

In [132]:
mypath = '/Users/arleenlindenmeyer/Desktop/ADS/data_mining/exam2/2000_2020/'

filenames_00_20 = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [133]:
filenames_00_20

['HangmansCurseTrailer.mp4',
 'TheDyingGaulTrailer.mp4',
 'NorbitTrailer.mp4',
 'FlightplanTrailer.mp4',
 '2012Trailer.mp4']

In [163]:
list_dfs_00_20 = []

for i in filenames_00_20:
    filename = i
    videopath = '/Users/arleenlindenmeyer/Desktop/ADS/data_mining/exam2/2000_2020/'+i
    df = process_video(video_path= videopath, filename=filename)
    list_dfs_00_20.append(df)

In [164]:
df_00_20 = pd.concat(list_dfs_00_20)

In [165]:
df_00_20.reset_index(drop=True, inplace=True)
df_00_20.head()

,video,path,scene,frame
0,HangmansCurseTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,0,2000_2020/HangmansCurseTrailer.mp41.jpeg
1,HangmansCurseTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,1,2000_2020/HangmansCurseTrailer.mp42.jpeg
2,HangmansCurseTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,2,2000_2020/HangmansCurseTrailer.mp43.jpeg
3,HangmansCurseTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,3,2000_2020/HangmansCurseTrailer.mp44.jpeg
4,HangmansCurseTrailer.mp4,/Users/arleenlindenmeyer/Desktop/ADS/data_mini...,4,2000_2020/HangmansCurseTrailer.mp45.jpeg


In [166]:
len(df_00_20)

344

In [167]:
all_classes = []

for file in tqdm(df_00_20.frame): # change for each years
    
    classes = [0] * 183
    
    color_image = load_image_from_path(file, color_mode='rgb', target_size=(INPUT_SHAPE)).astype('uint8')
    color_image_batched = np.expand_dims(color_image, axis=0) 

    results = hub_model(color_image_batched) # input the image to the model
    
    detection_scores = results['detection_scores'].numpy()[0]
    detection_classes = results['detection_classes'].numpy()[0]


    indices = np.where(detection_scores > 0.7) # only select labels with p > 0.7
    detection_classes_sel = detection_classes[indices] # get the classes where p > 0.7
    
    detection_classes_sel_list = detection_classes_sel.astype(np.int32).tolist()
    
    
    
    for i in detection_classes_sel_list:    # iterate over list, get index
        classes[i] += 1 # add 1 on this specific index   
    
    all_classes.append(classes)

  0%|          | 0/344 [00:00<?, ?it/s]

In [168]:
df_classes_00_20 = pd.DataFrame(all_classes, columns=coco_classes)

In [169]:
df_classes_00_20.reset_index(drop=True, inplace=True)
df_classes_00_20.head()

,unlabeled,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,...,wall-other,wall-panel,wall-stone,wall-tile,wall-wood,water-other,waterdrops,window-blind,window-other,wood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [170]:
df_00_20_classes = pd.concat([df_00_20, df_classes_00_20], axis=1)

In [171]:
df_00_20_classes = df_00_20_classes.iloc[:, 4:]

In [172]:
mean_classes_00_20 = df_00_20_classes.mean(axis=0)
mean_classes_00_20 = pd.DataFrame(mean_classes_00_20, columns=['mean'])

In [173]:
normalized_mean_classes_00_20 = mean_classes_00_20['mean'] / len(df_00_20_classes)
normalized_mean_classes_00_20 = pd.DataFrame(normalized_mean_classes_00_20, columns=['mean'])
normalized_mean_classes_00_20.sort_values(ascending= False, by = 'mean').head(10)

,mean
person,0.002755
tv,0.000406
dog,0.000093
car,0.000085
bed,0.000059
chair,0.000051
bird,0.000051
cow,0.000042
tie,0.000025
bottle,0.000025


### Comparison

Function to display dataframes side by side, taken from: https://pretagteam.com/question/display-two-dataframes-side-by-side-in-pandas

In [146]:
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [157]:
n_m_classes_20_40 = normalized_mean_classes_20_40.sort_values(ascending= False, by = 'mean').head(15)
n_m_classes_20_40 = pd.DataFrame(n_m_classes_20_40, columns=['mean'])

In [158]:
n_m_classes_60_80 = normalized_mean_classes_60_80.sort_values(ascending= False, by = 'mean').head(15)
n_m_classes_60_80 = pd.DataFrame(n_m_classes_60_80, columns=['mean'])

In [174]:
n_m_classes_00_20 = normalized_mean_classes_00_20.sort_values(ascending= False, by = 'mean').head(15)
n_m_classes_00_20 = pd.DataFrame(n_m_classes_00_20, columns=['mean'])

In [175]:
display_side_by_side(n_m_classes_20_40, n_m_classes_60_80, n_m_classes_00_20, titles = ['1920-1940', '1960-1980', '2000-2020'])

,mean
person,0.030325
tie,0.001109
chair,0.001109
clock,0.000740
tv,0.000740
parking meter,0.000370
boat,0.000370
wine glass,0.000370
laptop,0.000370
car,0.000370


## Interpretation and conclusion

Comparing the top 15 coco classes of the years, they look quite similar. Not surprisingly, persons are the most common class in all the years. Ties were more popular in the older movies, which makes sense to me, as a lot of people were dressed quite formal in movies back then. A clock is quite common in all years, but this may also be due to the fact that often, a round object is recognized as a clock in object detection. Wine glasses were apparently more popular in older movies, more recent movies include more bottles. Some object(s) in the old movies is/are recognized as a laptop, but this cannot be correct. 
I find it surprising that 'laptop' or 'cell phone' do not appear as very common objects in the years 2000-2020, but 'keyboard' is indeed a common object in these years.
The results show that a lot of common objects stay the same over all decades. Especially the years 1960-1980 and 2000-2020 seem to be quite similar in terms of their most common objects.

However, the flaws of the object detection and the limited number of trailers skew the results, and any drawn conclusions are limited in their validity. 

This analysis is based on only 15 trailers (5 for each of the years) and a limited number of frames (52, 299 and 344, depending on the years). To get more accurate results it would be best to include both more trailers and maybe also more frames per scene. Running the code with more trailers and frames, however, would of course need more computational power.
Further, because coco classes include only a certain number of classes, only those objects can be identified (and some objects may also be misclassified because of this). Thus, it would be best to use a model with more classes, which also detects objects accurately.